In [ ]:
import sys
import numpy as np
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from statistics import mode
from sklearn.metrics  import precision_score

def get_most_frequent_link(X):
    cnt_solved = 0
    cnt_not_solved = 0
    for l in X:
        if l == 'solved':
            cnt_solved +=1
        elif l == 'not-solved':
            cnt_not_solved += 1

    if cnt_solved >= cnt_not_solved:
        return 'solved'
    else:
        return 'not-solved'

def get_metrics(y_true, y_pred, majority):
    return precision_score(y_true, y_pred, labels=['solved', 'not-solved'], pos_label = majority), accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, labels=['solved', 'not-solved'], pos_label = majority)   
    
def HPO_KG_embedding_random(dim, budget, fixed_target, algo, removal_mode, rand_n):
   
    # k_space = [50, 100, 150, 200]
    # losses = [ "pairwise", "self_adversarial", "nll"]
    # rates = [1e-3,1e-4]

    k_space = [50, 100, 150, 200]
    losses = [ "pairwise"]
    rates = [1e-3]
    
    text = ""
    text_file = open(f"../Results/HPO_random_subset/{algo}/{removal_mode}/rand_{rand_n}_trace_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}.txt", "w")
    
    for k in k_space:
        for loss in losses:
            for lr in rates:
                y_pred = np.load(f"../Results/HPO_random_subset/{algo}/{removal_mode}/rand_{rand_n}_test_preds_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_k_{k}_loss_{loss}_lr_{lr}.npy")
                y_true = np.load(f"../Results/HPO_random_subset/{algo}/{removal_mode}/rand_{rand_n}_test_true_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_k_{k}_loss_{loss}_lr_{lr}.npy")

                majority_link = get_most_frequent_link(y_true)
                gmean, acc, f1 = get_metrics(y_true, y_pred, majority_link)
                text_file.write("gmean = "+str(gmean) +", f1 = "+str(f1)+", acc = "+str(acc)+", k = "+str(k)+", loss = "+str(loss)+", lr = "+str(lr)+"\n")
                text_file.flush()
                print("gmean = "+str(gmean) +", f1 = "+str(f1)+", acc = "+str(acc)+", k = "+str(k)+", loss = "+str(loss)+", lr = "+str(lr)+"\n")

    text_file.close()

if __name__ == "__main__":
    dim = 5
    budgets = [2000, 5000, 10000, 50000]
    algos = ["modDE", 'modCMA']
    removal_modes = ["config", "problem"]
    rands = [1,2,3,4,5]
    for budget in budgets:
        for rand_n in rands:
            for algo in algos:
                for removal_mode in removal_modes:
                    HPO_KG_embedding_random(5, budget, 0.1, algo, removal_mode, rand_n)

In [ ]:
import numpy as np
import pandas as pd
def parse_random_subset_results_HPO(algo, mode, dim, budgets, target, rands=[1,2,3,4,5]):

    data = []
    for budget in budgets:
        arr_KG = []
        for rand in rands:
            with open(f"../Results/HPO_random_subset/{algo}/{mode}/rand_{rand}_trace_ComplEx_dim_{dim}_budget_{budget}_target_{target:.5f}.txt") as file:
                lines = [line.rstrip() for line in file]
            scores = []
            for line in lines:
                scores.append(float(line.split(",")[1].split("=")[1]))
            best_score = max(scores)
            arr_KG.append(float(best_score))
            # print(scores)  
            # print(best_score)

        # print(budget)
        # print(arr_KG)
        
        print(f'{float(np.mean(arr_KG)):.3f} ({float(np.std(arr_KG)):.3f})')
        # data.append(arr)
    
if __name__ == "__main__":
    dim = 5
    budgets = [2000, 5000, 10000, 50000]
    algos = ["modDE"]
    removal_mode = "problem"
    for algo in algos:
        parse_random_subset_results_HPO(algo, removal_mode, dim, budgets, 0.1)

In [ ]:
import sys
import numpy as np
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from statistics import mode
from imblearn.metrics import geometric_mean_score


def get_metrics(y_true, y_pred):
    return geometric_mean_score(y_true, y_pred, labels=['solved', 'not-solved'], pos_label = 'solved'), accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, labels=['solved', 'not-solved'], pos_label = 'solved')   
    
def HPO_KG_embedding(dim, budget, fixed_target, algo):
   
    # k_space = [50, 100, 150, 200]
    # losses = [ "pairwise", "self_adversarial", "nll"]
    # rates = [1e-3,1e-4]

    k_space = [50, 100, 150, 200]
    losses = [ "pairwise"]
    rates = [1e-3, 1e-4]
    
    text = ""
    text_file = open(f"../Results/HPO/{algo}/trace_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}.txt", "w")
    
    for k in k_space:
        for loss in losses:
            for lr in rates:
                y_pred = np.load(f"../Results/HPO/{algo}/test_preds_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_k_{k}_loss_{loss}_lr_{lr}.npy")
                y_true = np.load(f"../Results/HPO/{algo}/test_true_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_k_{k}_loss_{loss}_lr_{lr}.npy")
   
                gmean, acc, f1 = get_metrics(y_true, y_pred)
                text_file.write("gmean = "+str(gmean) +", f1 = "+str(f1)+", acc = "+str(acc)+", k = "+str(k)+", loss = "+str(loss)+", lr = "+str(lr)+"\n")
                text_file.flush()
                print("gmean = "+str(gmean) +", f1 = "+str(f1)+", acc = "+str(acc)+", k = "+str(k)+", loss = "+str(loss)+", lr = "+str(lr)+"\n")

    text_file.close()

if __name__ == "__main__":
    dim = 30
    budgets = [2000, 5000, 10000, 50000]
    algos = ["modCMA", 'modDE']
    targets  = [10, 1, 0.1]
    for budget in budgets:
        for target in targets:
            for algo in algos:
                HPO_KG_embedding(dim, budget, target, algo)

In [ ]:
import numpy as np
import pandas as pd
def parse_results_HPO(algo, mode, dim, budgets, target):

    data = []
    for budget in budgets:
        arr_KG = []
        with open(f"../Results/HPO/{algo}/trace_ComplEx_dim_{dim}_budget_{budget}_target_{target:.5f}.txt") as file:
            lines = [line.rstrip() for line in file]
        scores = []
        for line in lines:
            scores.append(float(line.split(",")[0].split("=")[1]))
        best_score = max(scores)
        arr_KG.append(float(best_score))
        
        print(f'{float(np.mean(arr_KG)):.3f} ({float(np.std(arr_KG)):.3f})')
        # data.append(arr)


def parse_results_HPO(algo,  dim, budgets, target):

    data = []
    for budget in budgets:
        with open(f"../Results/HPO/{algo}/trace_ComplEx_dim_{dim}_budget_{budget}_target_{target:.5f}.txt", "r") as file:
            lines = [line.rstrip() for line in file]
        scores = []
        for line in lines:
            scores.append(float(line.split(",")[1].split("=")[1]))
        KG_f1 = max(scores)
        
        print(f'{float(KG_f1):.3f},')
        # data.append(arr)
    
if __name__ == "__main__":
    dim = 5
    budgets = [2000, 5000, 10000, 50000]
    algos = ["modCMA"]
    targets = [1,0.1, 0.001]
    for target in targets:
        for algo in algos:
            parse_results_HPO(algo, dim, budgets, target)